In [1]:
import psycopg2
import pandas as pd
from random import randint
from sql_queries import drop_table_queries,create_table_queries
from faker import Faker
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('aws.cfg')
host = config.get("LONDON_REGION","host")
user = config.get("LONDON_REGION","user")
dbname = config.get("LONDON_REGION","dbname")
password = config.get("LONDON_REGION","password")

# Create Faker class

In [3]:
fake = Faker()

# Create database

In [4]:
def create_database():
    """
    -creates the database called hr
    -return the connection and cursor to hr
    """
    try:
        conn = psycopg2.connect("host=" + host + " user=" + user + " password=" + password)
    except psycopg2.Error as e:
        print(e)
        
    conn.set_session(autocommit=True)
    curr = conn.cursor()
    
    curr.execute("drop database if exists hr")
    curr.execute("create database hr")
    
    curr.close()
    conn.close()
    
    conn_hr = psycopg2.connect("host=" + host + " user=" + user + " password=" + password + " dbname=" + dbname)
    cur_hr = conn_hr.cursor()
    
    return conn_hr, cur_hr

In [5]:
def drop_tables(conn,cur):
    """
    --Drop all the tables in the database
    """
    for query in drop_table_queries:
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e:
            print("Error : Dropping table failed for" + query)
            print(e)

In [6]:
def create_tables(conn, cur):
    """
    --Create the tables using create_table_queries 
    """
    for query in create_table_queries:
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e:
            print("Error : Create table failed for " + query)
            print(e)

In [7]:
conn,cur = create_database()
drop_tables(conn,cur)
create_tables(conn,cur)

In [8]:
fake_data_India= Faker("en_IN")

In [9]:
for i in range (10):
    CityID =fake_data_India.random_int()
    CityName = fake_data_India.city_name()
    State = fake_data_India.state()
    Pincode = fake_data_India.postcode()
    try:
        cur.execute("INSERT INTO City (CityID,CityName,State,Pincode) VALUES (%s, %s, %s, %s)", (CityID,CityName,State,Pincode))
    except psycopg2.Error as e:
        print(e)

In [10]:
try:
    cur.execute("select * from City")
except psycopg2.Error as e:
    print(e)

row = cur.fetchall()

In [11]:
row

[(6430, 'Sirsa', 'Nagaland', '979194'),
 (2909, 'Udaipur', 'Uttarakhand', '678007'),
 (1120, 'Hosur', 'Maharashtra', '799354'),
 (654, 'Kurnool', 'Arunachal Pradesh', '260309'),
 (6414, 'Amaravati', 'Bihar', '349338'),
 (8264, 'Thrissur', 'Punjab', '654740'),
 (9429, 'Madhyamgram', 'Kerala', '002530'),
 (5544, 'Thiruvananthapuram', 'Chhattisgarh', '922837'),
 (8737, 'Kumbakonam', 'Meghalaya', '737623'),
 (5021, 'Aurangabad', 'Sikkim', '023259')]

In [12]:
city_df = pd.DataFrame(row, columns =['CItyID', 'CityName','State','Pincode'])
city_df

,CItyID,CityName,State,Pincode
0,6430,Sirsa,Nagaland,979194
1,2909,Udaipur,Uttarakhand,678007
2,1120,Hosur,Maharashtra,799354
3,654,Kurnool,Arunachal Pradesh,260309
4,6414,Amaravati,Bihar,349338
5,8264,Thrissur,Punjab,654740
6,9429,Madhyamgram,Kerala,002530
7,5544,Thiruvananthapuram,Chhattisgarh,922837
8,8737,Kumbakonam,Meghalaya,737623
9,5021,Aurangabad,Sikkim,023259


In [13]:
for i in range (10):
    employeeid=fake_data_India.random_int()
    employeefirstname = fake_data_India.first_name()
    employeelastname=fake_data_India.last_name() 
    employeecityID = fake_data_India.random_int()
    try:
        cur.execute("INSERT INTO employee(employeeid,employeefirstname,employeelastname,employeecityID) VALUES (%s, %s, %s, %s)", (employeeid,employeefirstname,employeelastname,employeecityID))
    except psycopg2.Error as e:
        print(e)

In [14]:
try:
    cur.execute("select * from employee")
except psycopg2.Error as e:
    print(e)

row1 = cur.fetchall()

In [15]:
row1

[(5463, 'Stuvan', 'Rajagopal', 8839),
 (5261, 'Kashvi', 'Jain', 3830),
 (4244, 'Umang', 'Batta', 552),
 (2370, 'Nirvi', 'Sachar', 3328),
 (8663, 'Indranil', 'Ben', 8629),
 (5967, 'Kavya', 'Venkatesh', 7048),
 (5677, 'Anika', 'Char', 9009),
 (5063, 'Adira', 'Goel', 9096),
 (4905, 'Tara', 'Mane', 3899),
 (1148, 'Aaina', 'Chaudhuri', 1934)]

In [16]:
employee_df = pd.DataFrame(row1, columns =['employeeid','employeefirstname','employeelastname','employeecityID'])
employee_df

,employeeid,employeefirstname,employeelastname,employeecityID
0,5463,Stuvan,Rajagopal,8839
1,5261,Kashvi,Jain,3830
2,4244,Umang,Batta,552
3,2370,Nirvi,Sachar,3328
4,8663,Indranil,Ben,8629
5,5967,Kavya,Venkatesh,7048
6,5677,Anika,Char,9009
7,5063,Adira,Goel,9096
8,4905,Tara,Mane,3899
9,1148,Aaina,Chaudhuri,1934
